In [1]:
import csv # module for handling CSV file input/output
import pandas as pd
import numpy as np
import sympy as sp
import unittest
from decimal import Decimal

In [2]:
def read_csv_file(filepath, x_col: int, y_col: int, x_err_col: None, y_err_col: None):
    # create empty lists to store x and y values
    x_data = []
    y_data = []

    # open the CSV file in read mode
    # newline='' prevents issues with line breaks across operating systems
    with open(filepath, newline='') as file:
        reader = csv.reader(file)  # turn the file into a CSV reader object

        # read header row to get column titles
        header = next(reader, None)
        x_title = header[x_col - 1]  # title of x column
        y_title = header[y_col - 1]  # title of y column

        # go through each row in the file
        for row in reader:
            # take the value from the chosen x column, convert to float, add to list
            x_data.append(float(row[x_col - 1]))
            # take the value from the chosen y column, convert to float, add to list
            y_data.append(float(row[y_col - 1]))


        # self.x_values = x_data
        # self.y_values = y_data
        # self.x_error = find_error(x_data, x_err_col)
        # self.y_error = find_error(y_data, y_err_col)
        # self.x_title = x_title
        # self.y_title = y_title
    return

In [3]:
def read_excel(filepath, x: int, y: int, x_err_col: None, y_err_col: None):
    # read the Excel file into a DataFrame
    df = pd.read_excel(filepath)

    # extract values from the chosen x column (1-based index)
    # convert each value to int if it is already an int, otherwise to float
    x_data = [int(val) if isinstance(val, int) else float(val) for val in df.iloc[:, x - 1]]

    # extract values from the chosen y column (1-based index)
    # same conversion logic as for x values
    y_data = [int(val) if isinstance(val, int) else float(val) for val in df.iloc[:, y - 1]]

    # get the column titles (names of x and y columns)
    x_title, y_title = df.columns

        # self.x_values = x_data
        # self.y_values = y_data
        # self.x_error = find_error(x_data, x_err_col)
        # self.y_error = find_error(y_data, y_err_col)
        # self.x_title = x_title
        # self.y_title = y_title
    return

In [4]:
x, y = sp.symbols('x y')

def linearise(equation):

    """
    Linearise common non-linear functions for straight-line graphs.

    Supported transformations:
    - Exponential: y = a*exp(b*x) + c -> ln(y - c) = ln(a) + b*x
    - Reciprocal: y = a/x + c -> remains y = a/x + c (already linear in 1/x)
    - Power/Polynomial: y = a*x^n + c -> remains unchanged (linear in x^n)

    Accepts equations in the form of SymPy Eq objects or expressions (assumes = 0).
    Handles cases where y is not isolated, such as:
    - 2*y = x^2 + 3
    - y^2 = x + c
    - a*y^n = b*x^m + c

    All other forms are kept unchanged as they're already
    linear in the transformed variables.
    """

    # Convert to equation if just an expression is passed
    if not isinstance(equation, sp.Eq):
        expr = equation
        # Try to determine if it's meant to be y = expr or expr = 0
        if y in expr.free_symbols:
            # Check if it's already solved for y
            if expr.is_Add or expr.is_Mul or expr.is_Pow:
                equation = sp.Eq(y, expr)
            else:
                equation = sp.Eq(expr, 0)
        else:
            # No y in expression, treat as y = expr
            equation = sp.Eq(y, expr)

    lhs = equation.lhs
    rhs = equation.rhs

    # Determine which side contains y and which contains the main expression
    if y in lhs.free_symbols and y not in rhs.free_symbols:
        y_side = lhs
        expr_side = rhs
    elif y in rhs.free_symbols and y not in lhs.free_symbols:
        # Swap to keep y on left
        y_side = rhs
        expr_side = lhs
    else:
        # Both sides have y or neither side has y, return unchanged
        return equation

    # Check if the expression side (without y) has exponential
    if expr_side.has(sp.exp):
        c, rest = expr_side.as_coeff_Add()
        if rest.has(sp.exp):
            coeff, exp_term = rest.as_coeff_Mul()
            if isinstance(exp_term, sp.exp):
                b = exp_term.args[0]
            else:
                b = 1
            # Apply transformation: ln(y_side - c) = ln(coeff) + b
            return sp.Eq(sp.log(y_side - c), sp.log(coeff) + b)
        else:
            return sp.Eq(y_side, expr_side)

    # Check for reciprocal in expression side
    if expr_side.has(1/x):
        return sp.Eq(y_side, expr_side)

    # All other cases (power, polynomial, linear, or y^n forms) -> keep unchanged
    # This includes: y = x^n, 2*y = x^2, y^2 = x + c, etc.
    return sp.Eq(y_side, expr_side)


In [5]:
class TestLinearise(unittest.TestCase):
    def test_exp(self):
        expr = sp.exp(x)
        expected = sp.Eq(sp.log(y), x)
        self.assertEqual(linearise(expr), expected)

    def test_scaled_exp(self):
        expr = 2*sp.exp(3*x)
        expected = sp.Eq(sp.log(y), sp.log(2) + 3*x)
        self.assertEqual(linearise(expr), expected)

    def test_power(self):
        expr = x**2
        expected = sp.Eq(y, expr)
        self.assertEqual(linearise(expr), expected)

    def test_linear(self):
        expr = 3*x + 5
        expected = sp.Eq(y, 3*x + 5)
        self.assertEqual(linearise(expr), expected)

    def test_fraction(self):
        expr = 4/x
        expected = sp.Eq(y, expr)
        self.assertEqual(linearise(expr), expected)

    def test_fraction_exp(self):
        expr = 4/x**2
        expected = sp.Eq(y, expr)
        self.assertEqual(linearise(expr), expected)

    def test_negative_exp(self):
        expr = 5*sp.exp(-x/3)
        expected = sp.Eq(sp.log(y), sp.log(5) - x/3)
        self.assertEqual(linearise(expr), expected)

    def test_polynomial_with_constant(self):
        expr = x**2 + 3
        expected = sp.Eq(y, x**2 + 3)
        self.assertEqual(linearise(expr), expected)

    def test_equation_scaled_y(self):
        equation = sp.Eq(2*y, x**2 + 3)
        expected = sp.Eq(2*y, x**2 + 3)
        self.assertEqual(linearise(equation), expected)

    def test_equation_y_squared(self):
        equation = sp.Eq(y**2, x + 5)
        expected = sp.Eq(y**2, x + 5)
        self.assertEqual(linearise(equation), expected)

    def test_equation_y_cubed(self):
        equation = sp.Eq(y**3, 2*x**2 + 1)
        expected = sp.Eq(y**3, 2*x**2 + 1)
        self.assertEqual(linearise(equation), expected)

    def test_reciprocal_with_constant(self):
        expr = 3/x + 2
        expected = sp.Eq(y, expr)
        self.assertEqual(linearise(expr), expected)

    def test_scaled_power(self):
        expr = 5*x**3
        expected = sp.Eq(y, expr)
        self.assertEqual(linearise(expr), expected)

    def test_equation_reversed(self):
        equation = sp.Eq(x**2, y)
        expected = sp.Eq(y, x**2)
        self.assertEqual(linearise(equation), expected)

suite = unittest.TestLoader().loadTestsFromTestCase(TestLinearise)
unittest.TextTestRunner(verbosity=2).run(suite)

test_equation_reversed (__main__.TestLinearise.test_equation_reversed) ... ok
test_equation_scaled_y (__main__.TestLinearise.test_equation_scaled_y) ... ok
test_equation_y_cubed (__main__.TestLinearise.test_equation_y_cubed) ... ok
test_equation_y_squared (__main__.TestLinearise.test_equation_y_squared) ... ok
test_exp (__main__.TestLinearise.test_exp) ... ok
test_fraction (__main__.TestLinearise.test_fraction) ... ok
test_fraction_exp (__main__.TestLinearise.test_fraction_exp) ... ok
test_linear (__main__.TestLinearise.test_linear) ... ok
test_negative_exp (__main__.TestLinearise.test_negative_exp) ... ok
test_polynomial_with_constant (__main__.TestLinearise.test_polynomial_with_constant) ... ok
test_power (__main__.TestLinearise.test_power) ... ok
test_reciprocal_with_constant (__main__.TestLinearise.test_reciprocal_with_constant) ... ok
test_scaled_exp (__main__.TestLinearise.test_scaled_exp) ... ok
test_scaled_power (__main__.TestLinearise.test_scaled_power) ... ok

---------------

<unittest.runner.TextTestResult run=14 errors=0 failures=0>

In [6]:
def resolution(num):
    d = Decimal(str(num))
    return Decimal(f'1e{d.as_tuple().exponent}')


def find_error(inputs:list, axis_err = None):
    axis = np.array(inputs)
    if axis_err is not None:
        error = np.array(axis_err)
    else:
        res = []
        for i in axis:
            res.append(resolution(i))
        error = np.full_like(axis, min(res), dtype='float')
    return error

